## 2023 Open Science Data Challenge - Sentinel-2 Phenology with Cloud Filtering

This notebook calculates vegetation phenology using Sentinel-2 data with cloud filtering. This pixel-based approach is better than a scene-based approach since clouds are quite random for any given time and location. To address phenology, the algorithm uses the Normalized Difference Vegetation Index (NDVI) which is a common proxy for vegetation growth and health. The outputs of this notebook can be used to assess differences in agriculture fields over time or space and also allow the assessment of growing states such as planting and harvesting. The baseline data is [Sentinel-2 Level-2A](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) data from the MS Planetary Computer catalog.

In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba


from tqdm import tqdm
tqdm.pandas()

### Load the satellite data

First, we define our area of interest using latitude and longitude coordinates of the centroid. Then we define the size of the surrounding bounding box (in degrees). GeoJSON format uses a specific order: (longitude, latitude), so be careful when entering the coordinates. Finally, we define the time window consistent with a typical rice growing season.

In [2]:
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [3]:
def bounding_box(lat_long):
    box_size_deg = 0.10 

    min_lon = lat_long[1]-box_size_deg/2
    min_lat = lat_long[0]-box_size_deg/2
    max_lon = lat_long[1]+box_size_deg/2
    max_lat = lat_long[0]+box_size_deg/2

    bbox = (min_lon, min_lat, max_lon, max_lat)
    return bbox

In [4]:
def unfiltered_DataFrame(latlong, xx):
    xx = xx.mean(dim=['latitude','longitude']).compute()
    n = len(xx['red'])

    time = xx.coords['time'].values
    red = xx['red'].values.tolist()
    green = xx['green'].values.tolist()
    blue = xx['blue'].values.tolist()
    nir = xx['nir'].values.tolist()

    df = pd.DataFrame(data=list(zip([latlong]*n, time, red, green, blue, nir)), columns=['Latitude and Longitude', 'time', 'red_o', 'green_o', 'blue_o', 'nir_o'])
    return df

In [5]:
def filtered_DataFrame(latlong, xx):
    cloud_mask = (xx.SCL != 0) & (xx.SCL != 1) & (xx.SCL != 3) & (xx.SCL != 6) & (xx.SCL != 8) & (xx.SCL != 9) & (xx.SCL != 10) 
    xx = xx.where(cloud_mask).astype("uint16")
    
    xx = xx.mean(dim=['latitude','longitude']).compute()
    n = len(xx['red'])

    time = xx.coords['time'].values
    red = xx['red'].values.tolist()
    green = xx['green'].values.tolist()
    blue = xx['blue'].values.tolist()
    nir = xx['nir'].values.tolist()

    df = pd.DataFrame(data=list(zip([latlong]*n, time, red, green, blue, nir)), columns=['Latitude and Longitude', 'time', 'red_f', 'green_f', 'blue_f', 'nir_f'])
    return df

In [6]:
def get_sentinel_2_data(latlong, time_slice):
    
    latlong_=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    latlong_ = list(map(lambda x: float(x), latlong_))
    bbox_of_interest = bounding_box(latlong_)

    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bbox_of_interest, datetime=time_window)
    items = list(search.get_all_items())
    
    resolution = 20  # meters per pixel 
    scale = resolution / 111320.0 # degrees per pixel for CRS:4326 
    
    
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL"],
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bbox_of_interest
    )
    
    xx_o = unfiltered_DataFrame(latlong, xx)
    xx_f = filtered_DataFrame(latlong, xx)
    
    data = xx_o.merge(xx_f, on=['Latitude and Longitude', 'time'], how='inner')
    return data

In [13]:
time_window="2021-11-01/2022-08-31"
data = pd.DataFrame(columns=['Latitude and Longitude', 'time', 'red_o', 'green_o', 'blue_o', 'nir_o', 'red_f', 'green_f', 'blue_f', 'nir_f'])
    
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    df = get_sentinel_2_data(coordinates,time_window)
    data = pd.concat([data, df], ignore_index=True)
    #xx = get_sentinel_2_data('(10.4391, 105.3338)',time_window)
    

100%|██████████| 600/600 [3:05:13<00:00, 18.52s/it]  


In [14]:
data['NDVI_o'] = (data['nir_o']-data['red_o']) / (data['nir_o']+data['red_o'])
data['NDVI_f'] = (data['nir_f']-data['red_f']) / (data['nir_f']+data['red_f'])

data['EVI_o'] = 2.5 * ((data['nir_o']-data['red_o']) / (data['nir_o']+6*data['red_o']-7.5*data['blue_o']+1)) #C1=6, C2=7.5, L=1, G=2.5
data['EVI_f'] = 2.5 * ((data['nir_f']-data['red_f']) / (data['nir_f']+6*data['red_f']-7.5*data['blue_f']+1)) #C1=6, C2=7.5, L=1, G=2.5

data['SAVI_o'] = ((data['nir_o']-data['red_o']) / (data['nir_o']+data['red_o']+0.5)) * (1+0.5) #L-0.5
data['SAVI_f'] = ((data['nir_f']-data['red_f']) / (data['nir_f']+data['red_f']+0.5)) * (1+0.5) #L-0.5

In [15]:
data.shape

(33249, 16)

In [17]:
data = data.merge(crop_presence_data, on='Latitude and Longitude', how='inner')

In [18]:
data.shape

(33249, 17)

In [24]:
data['time'] = pd.to_datetime(data['time']).dt.date

In [25]:
data.head()

,Latitude and Longitude,time,red_o,green_o,blue_o,nir_o,red_f,green_f,blue_f,nir_f,NDVI_o,NDVI_f,EVI_o,EVI_f,SAVI_o,SAVI_f,Class of Land
0,"(10.323727047081501, 105.2516346045924)",2021-11-11,7819.483755,8422.723397,9346.066099,7747.999305,0.000000,0.000000,0.000000,0.000000,-0.004592,NaN,0.011582,0.000000,-0.006888,0.000000,Rice
1,"(10.323727047081501, 105.2516346045924)",2021-11-16,7220.483762,7529.236295,8033.924712,7543.402830,0.000000,0.000000,0.000000,0.000000,0.021872,NaN,-0.086000,0.000000,0.032807,0.000000,Rice
2,"(10.323727047081501, 105.2516346045924)",2021-11-21,785.254969,909.071122,572.435513,3076.932723,711.999028,829.774052,497.177603,2958.218390,0.593363,0.612013,1.638694,1.603352,0.889929,0.917894,Rice
3,"(10.323727047081501, 105.2516346045924)",2021-11-26,853.906450,855.202647,474.339289,2688.530016,847.737325,849.601317,470.856904,2684.666110,0.517899,0.520022,1.077815,1.082925,0.776739,0.779923,Rice
4,"(10.323727047081501, 105.2516346045924)",2021-12-01,5755.051453,6113.884475,6647.837294,5842.471497,0.038970,0.046164,0.057875,0.046357,0.007538,0.086576,-0.023042,0.021827,0.011306,0.018931,Rice


In [26]:
data.to_csv('Sentinel-2-SCL.csv', index = False)